In [3]:
! pip install nlpsandbox-client

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 97 kB 3.1 MB/s eta 0:00:011
     |████████████████████████████████| 97 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 247 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 3.8 MB/s eta 0:00:011
  Created wheel for nlpsandbox-client: filename=nlpsandbox_client-4.2.0-py3-none-any.whl size=358102 sha256=8076f7f7e4038a5d0fbb3aea7ac9f9c1925e8202f3927c5d71b6a61b44fca668
  Stored in directory: /home/nlp/.cache/pip/wheels/f5/9b/21/d3ae0c553bce8ffdc272ec6b79e1fc58d990cd9e4c6d7b0b0f
  Created wheel for secretstorage: filename=SecretStorage-2.3.1-py3-none-any.whl size=13961 sha25

In [4]:
from nlpsandboxclient import client

In [7]:
data_node_host = "http://data-node-nginx:80/api/v1"
annotator_host = "http://phi-annotator-nginx:80/api/v1"

## Load the dataset

In [8]:
datasets = client.list_datasets(host=data_node_host)

for dataset in datasets:
      print(dataset)

{'name': 'datasets/i2b2-test-dataset'}


## Test annotator 

In [9]:
# an example of using the annotator to annotate one note
example_note = {
   "identifier": "note-1",
   "type": "loinc:LP29684-5",
   "patientId": "507f1f77bcf86cd799439011",
   "text": "On 12/26/2020, Ms. Chloe Price met with Dr. Prescott in Seattle."
}

annotations = client.annotate_note(host=annotator_host,
                            note=example_note,
                            tool_type="nlpsandbox:date-annotator")

print(annotations)

{'textDateAnnotations': [{'start': 3, 'length': 10, 'text': '12/26/2020', 'confidence': 95.5, 'dateFormat': 'MM/DD/YYYY'}, {'start': 9, 'length': 4, 'text': '2020', 'confidence': 95.5, 'dateFormat': 'YYYY'}]}


In [10]:
# an example of using the annotator to annotate all notes in the test dataset
from nlpsandboxclient import client
notes = client.list_notes(host=data_node_host,
                  dataset_id="i2b2-test-dataset",
                  fhir_store_id="evaluation")

test_notes = list(notes)
print(len(test_notes))
annotation_store_name = 'sparknlp'
dataset_name = 'i2b2-test-dataset'
annotation_list = []
converted_annotation_list = []
for i in test_notes:
    note_name= i.pop('note_name', None)
    note_id = note_name[-6:]
    # There is an error in annotation generated from  client.annotate_note, the note_name is "dataset/.."  but it should be "datasets/.."
    note_name = "datasets"+note_name[7:]
    annotations = client.annotate_note(host=annotator_host,
                            note=i,
                            tool_type="nlpsandbox:date-annotator")
    #annotation_name = f"datasets/{dataset_name}/annotationStores/{annotation_store_name}/annotations/{note_id}"
    converted_annotation = {"annotationSource":{"resourceSource":{"name":note_name}},"textContactAnnotations": [], "textCovidSymptomAnnotations":[],"textDateAnnotations":annotations["textDateAnnotations"],"textIdAnnotations":[],"textLocationAnnotations": [],"textPersonNameAnnotations": []}
    converted_annotation_list.append(converted_annotation)

514


In [8]:
converted_annotation_list

[]

In [15]:
## store the annotation to the data node
client.store_annotations(host=data_node_host,
                 dataset_id=dataset_name,
                 annotation_store_id= annotation_store_name,
                 annotations = converted_annotation_list,
                 delete_existing_annotations = True)

Deleted existing Annotation Store
Created Annotation Store
